In [3]:
import youtube_dl
import os

ydl_opts={}
with youtube_dl.YoutubeDL(ydl_opts) as ydl:
    ydl.download(['https://www.youtube.com/watch?v=3vPdtajOJfw'])

[youtube] 3vPdtajOJfw: Downloading webpage
[youtube] 3vPdtajOJfw: Downloading MPD manifest
[download] Destination: Focusing on the Economic Priorities for the Middle Class Nationwide-3vPdtajOJfw.mp4
[download] 100% of 31.52MiB in 09:5991KiB/s ETA 00:004


In [7]:
import mmcv
import numpy as np
import os.path as osp
from PIL import Image

video_path = 'Focusing on the Economic Priorities for the Middle Class Nationwide-3vPdtajOJfw.mp4'
video = mmcv.VideoReader(video_path)
print(video.width, video.height, video.resolution, len(video))
video = video[1000:1100]

os.makedirs(osp.join('obama', 'hq_raw'), exist_ok=True)

for i, frame in enumerate(video):
    hq_raw = np.flip(frame, axis=2) / 255
    hq_raw = Image.fromarray((hq_raw * 255).astype(np.uint8))
    frame_idx = i + 1000
    hq_raw.save(osp.join('obama', 'hq_raw', f'{frame_idx:04}.png'))


1280 720 (1280, 720) 6919


In [6]:
import glob
import dlib
import cv2
import os.path as osp
import sys
sys.path.append('/home/jinsuyoo/ResizeRight')
from resize_right import resize
import numpy as np

detector = dlib.get_frontal_face_detector() 
#sp = dlib.shape_predictor("shape_predictor_5_face_landmarks.dat") 
#facerec = dlib.face_recognition_model_v1("dlib_face_recognition_resnet_model_v1.dat")

img_path = sorted(glob.glob('obama/hq_raw/*.png'))
os.makedirs(osp.join('obama', 'lq'), exist_ok=True)

'''
for i, ip in enumerate(img_path):
    img = dlib.load_rgb_image(ip)
    dets = detector(img, 1)[0]
    crop = img[dets.top():dets.bottom(), dets.left():dets.right()]
    crop = np.clip(resize(crop, out_shape=(64,64)), 0, 255)
    frame_idx = i + 1000 
    crop = np.flip(crop, axis=2)
    cv2.imwrite(osp.join('obama', 'lq', f'{frame_idx:04}.png'), crop)
'''

avg_top, avg_bottom, avg_left, avg_right = [], [] ,[] ,[]
padding = 50


for ip in img_path:
    img = dlib.load_rgb_image(ip)
    dets = detector(img, 1)[0]
    avg_top.append(dets.top())
    avg_bottom.append(dets.bottom())
    avg_left.append(dets.left())
    avg_right.append(dets.right())
avg_top = int(np.mean(np.array(avg_top)))
avg_bottom = int(np.mean(np.array(avg_bottom)))
avg_left = int(np.mean(np.array(avg_left)))
avg_right = int(np.mean(np.array(avg_right)))
print(avg_top, avg_bottom, avg_left, avg_right)
input()

start_img_path = img_path[0]
start_img = dlib.load_rgb_image(start_img_path)
dets = detector(start_img, 1)[0]

for i, ip in enumerate(img_path):
    img = dlib.load_rgb_image(ip)
    crop = img[avg_top-padding:avg_bottom+padding, avg_left-padding:avg_right+padding]
    crop = np.clip(resize(crop, out_shape=(64,64)), 0, 255)
    frame_idx = i + 1000 
    crop = np.flip(crop, axis=2)
    cv2.imwrite(osp.join('obama', 'lq', f'{frame_idx:04}.png'), crop)


193 486 438 731
